In [ ]:
# =========================================
# Phase 4 — 7-day Multi-step + Uncertainty + Classification
# + Report generator -> artifacts/phase4/
# =========================================

import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------
# A) Config
# -------------------------
PHASE2_DIR = "artifacts/phase2"
OUT_DIR = "artifacts/phase4"
os.makedirs(OUT_DIR, exist_ok=True)

TRAIN_F = os.path.join(PHASE2_DIR, "train_features.csv")
VAL_F   = os.path.join(PHASE2_DIR, "val_features.csv")
TEST_F  = os.path.join(PHASE2_DIR, "test_features.csv")
FEAT_F  = os.path.join(PHASE2_DIR, "feature_cols.json")
META_F  = os.path.join(PHASE2_DIR, "split_meta.json")

for p in [TRAIN_F, VAL_F, TEST_F, FEAT_F, META_F]:
    assert os.path.exists(p), f"Missing: {p}"

with open(FEAT_F, "r", encoding="utf-8") as f:
    feature_cols = json.load(f)["feature_cols"]

with open(META_F, "r", encoding="utf-8") as f:
    split_meta = json.load(f)

if "Store" in feature_cols:
    print("WARNING: 'Store' found in feature_cols.json — removing it.")
    feature_cols = [c for c in feature_cols if c != "Store"]

TARGET = "Sales"
META_COLS = ["Store", "Date"]


H_FULL = 7

HORIZONS = list(range(1, H_FULL + 1)) 

QUANTILES = [0.1, 0.5, 0.9]

Q_LR = 0.05
Q_EPOCHS = 18
Q_BATCH = 8192
Q_L2 = 1e-4

DO_ENSEMBLE = True
ENSEMBLE_HORIZON = 7    
ENSEMBLE_K = 7

CLASS_HORIZONS = [1, 7] 
CLF_LR = 0.2
CLF_EPOCHS = 22
CLF_BATCH = 8192
CLF_L2 = 1e-4

SAMPLE_STORES_N = 10
FORECAST_ORIGIN_MODE = "before_test" 
CUSTOM_ORIGIN_DATE = None 

# -------------------------
# B) Helpers: IO + plots
# -------------------------
def save_json(path, obj):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

def save_df_csv(path, df):
    df.to_csv(path, index=False)

def save_fig(path):
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()

save_json(os.path.join(OUT_DIR, "config_phase4.json"), {
    "H_FULL": H_FULL, "HORIZONS": HORIZONS,
    "QUANTILES": QUANTILES, "Q_LR": Q_LR, "Q_EPOCHS": Q_EPOCHS,
    "Q_BATCH": Q_BATCH, "Q_L2": Q_L2, "DO_ENSEMBLE": DO_ENSEMBLE,
    "ENSEMBLE_HORIZON": ENSEMBLE_HORIZON, "ENSEMBLE_K": ENSEMBLE_K,
    "CLASS_HORIZONS": CLASS_HORIZONS, "CLF_LR": CLF_LR,
    "CLF_EPOCHS": CLF_EPOCHS, "CLF_BATCH": CLF_BATCH, "CLF_L2": CLF_L2,
    "SAMPLE_STORES_N": SAMPLE_STORES_N, "FORECAST_ORIGIN_MODE": FORECAST_ORIGIN_MODE,
    "CUSTOM_ORIGIN_DATE": CUSTOM_ORIGIN_DATE, "split_meta": split_meta,
})

# -------------------------
# C) Load Phase2 CSVs
# -------------------------
NROWS_DEBUG = None 

train_df = pd.read_csv(TRAIN_F, nrows=NROWS_DEBUG)
val_df   = pd.read_csv(VAL_F,   nrows=NROWS_DEBUG)
test_df  = pd.read_csv(TEST_F,  nrows=NROWS_DEBUG)

def ensure_store_date(df):
    df["Date"] = pd.to_datetime(df["Date"])
    s = pd.to_numeric(df["Store"], errors="coerce")
    df["Store"] = np.round(s).astype(int)
    return df

train_df = ensure_store_date(train_df)
val_df   = ensure_store_date(val_df)
test_df  = ensure_store_date(test_df)

df_all = pd.concat([train_df, val_df, test_df], ignore_index=True)
df_all = df_all.sort_values(["Store","Date"]).reset_index(drop=True)

val_start  = pd.to_datetime(split_meta["val_start"])
test_start = pd.to_datetime(split_meta["test_start"])
max_date   = pd.to_datetime(split_meta["max_date"])

print("Split meta:", split_meta)
print("Total rows:", len(df_all), "| Num features:", len(feature_cols))

# -------------------------
# D) Metrics (FIXED FOR LOG-TARGET)
# -------------------------
def rmse(y_true, y_pred):
    y_true = np.expm1(y_true)
    y_pred = np.expm1(y_pred)
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def mape(y_true, y_pred):
    y_true = np.expm1(y_true)
    y_pred = np.expm1(y_pred)
    mask = y_true > 0
    if not np.any(mask): return 0.0
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])))

def rmspe(y_true, y_pred):
    y_true = np.expm1(y_true)
    y_pred = np.expm1(y_pred)
    mask = y_true > 0
    if not np.any(mask): return 0.0
    return float(np.sqrt(np.mean(((y_true[mask] - y_pred[mask]) / y_true[mask]) ** 2)))

def r2(y_true, y_pred):
    y_true = np.expm1(y_true)
    y_pred = np.expm1(y_pred)
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2) + 1e-12
    return float(1 - ss_res / ss_tot)

def pinball_loss(y, yhat, q):
    y = np.expm1(y)
    yhat = np.expm1(yhat)
    e = y - yhat
    return float(np.mean(np.maximum(q*e, (q-1)*e)))

# -------------------------
# E) Build time-safe masks per horizon
# -------------------------
def split_masks_for_h(df, h):
    train_end = val_start - pd.Timedelta(days=h)
    val_end   = test_start - pd.Timedelta(days=h)
    test_end  = max_date - pd.Timedelta(days=h)

    m_train = (df["Date"] <= train_end)
    m_val   = (df["Date"] >= val_start) & (df["Date"] < test_start) & (df["Date"] <= val_end)
    m_test  = (df["Date"] >= test_start) & (df["Date"] <= test_end)
    return m_train, m_val, m_test

def build_xy_for_h(df, h, mask):
    sub = df.loc[mask, :].copy()
    sub = sub.sort_values(["Store","Date"])
    sub["y"] = sub.groupby("Store", sort=False)[TARGET].shift(-h)
    sub = sub.dropna(subset=["y"])
    X = sub[feature_cols].to_numpy(dtype=np.float32)
    y = sub["y"].to_numpy(dtype=np.float32)
    meta = sub[META_COLS].copy()
    return X, y, meta

# -------------------------
# F) Quantile Regression model (from scratch)
# -------------------------
class QuantileLinearGD:
    def __init__(self, q=0.5, lr=0.05, epochs=20, batch_size=8192, l2=1e-4, seed=42, verbose=0):
        self.q=q; self.lr=lr; self.epochs=epochs; self.batch_size=batch_size
        self.l2=l2; self.seed=seed; self.verbose=verbose
        self.w=None; self.b=0.0

    def fit(self, X, y, X_val=None, y_val=None):
        n, d = X.shape
        rng = np.random.default_rng(self.seed)
        self.w = rng.normal(0, 0.01, size=(d,)).astype(np.float32)
        self.b = 0.0

        best = (float("inf"), self.w.copy(), float(self.b))

        for ep in range(1, self.epochs+1):
            idx = rng.permutation(n)
            Xs, ys = X[idx], y[idx]

            for i in range(0, n, self.batch_size):
                xb = Xs[i:i+self.batch_size]
                yb = ys[i:i+self.batch_size]

                pred = xb @ self.w + self.b
                e = yb - pred

                grad_pred = np.where(e >= 0, -self.q, -(self.q - 1.0)).astype(np.float32)

                gw = (xb.T @ grad_pred) / len(xb)
                gb = float(np.mean(grad_pred))

                if self.l2 > 0:
                    gw = gw + self.l2 * self.w

                self.w -= self.lr * gw
                self.b -= self.lr * gb

            if X_val is not None and self.verbose:
                pv = self.predict(X_val)
                lv = pinball_loss(y_val, pv, self.q)
                if lv < best[0]:
                    best = (lv, self.w.copy(), float(self.b))
                print(f"Epoch {ep:02d} | Val pinball(q={self.q})={lv:.5f}")

        if X_val is not None:
            _, self.w, self.b = best
        return self

    def predict(self, X):
        return (X @ self.w + self.b).astype(np.float32)

# -------------------------
# G) Train multi-horizon quantile models + save metrics
# -------------------------
models_q = {}
rows = []

for h in HORIZONS:
    m_tr, m_va, m_te = split_masks_for_h(df_all, h)
    Xtr, ytr, _ = build_xy_for_h(df_all, h, m_tr)
    Xva, yva, _ = build_xy_for_h(df_all, h, m_va)
    Xte, yte, _ = build_xy_for_h(df_all, h, m_te)

    print(f"\n=== Horizon h={h} days ===")
    print("Train/Val/Test:", Xtr.shape, Xva.shape, Xte.shape)

    models_q[h] = {}
    for q in QUANTILES:
        mdl = QuantileLinearGD(q=q, lr=Q_LR, epochs=Q_EPOCHS, batch_size=Q_BATCH, l2=Q_L2, seed=42, verbose=0)
        mdl.fit(Xtr, ytr, Xva, yva)
        models_q[h][q] = mdl

    p10_v = models_q[h][0.1].predict(Xva)
    p50_v = models_q[h][0.5].predict(Xva)
    p90_v = models_q[h][0.9].predict(Xva)

    p50_t = models_q[h][0.5].predict(Xte)

    yva_real = np.expm1(yva)
    p10_v_real = np.expm1(p10_v)
    p90_v_real = np.expm1(p90_v)
    
    coverage_v = float(np.mean((yva_real >= p10_v_real) & (yva_real <= p90_v_real)))
    width_v = float(np.mean(p90_v_real - p10_v_real))

    rows.append({
        "horizon_days": h,
        "val_rmse_p50": rmse(yva, p50_v),
        "val_rmspe_p50": rmspe(yva, p50_v), 
        "val_mape_p50": mape(yva, p50_v),
        "val_r2_p50": r2(yva, p50_v),
        "val_pinball_p10": pinball_loss(yva, p10_v, 0.1),
        "val_pinball_p50": pinball_loss(yva, p50_v, 0.5),
        "val_pinball_p90": pinball_loss(yva, p90_v, 0.9),
        "val_p10p90_coverage": coverage_v,
        "val_p10p90_width": width_v,
        "test_rmse_p50": rmse(yte, p50_t),
        "test_rmspe_p50": rmspe(yte, p50_t), 
        "test_mape_p50": mape(yte, p50_t),
        "test_r2_p50": r2(yte, p50_t),
    })

metrics_q = pd.DataFrame(rows).sort_values("horizon_days")
save_df_csv(os.path.join(OUT_DIR, "metrics_multistep_quantile.csv"), metrics_q)
display(metrics_q.head(7))

# --- Plots ---
plt.figure(figsize=(8,3))
plt.plot(metrics_q["horizon_days"], metrics_q["val_rmspe_p50"], marker="o", label="VAL RMSPE (p50)")
plt.plot(metrics_q["horizon_days"], metrics_q["test_rmspe_p50"], marker="o", label="TEST RMSPE (p50)")
plt.title("RMSPE vs Horizon (Direct multi-step, 7 days)")
plt.xlabel("Horizon (days ahead)")
plt.ylabel("RMSPE")
plt.legend()
save_fig(os.path.join(OUT_DIR, "rmspe_vs_horizon.png"))

plt.figure(figsize=(8,3))
plt.plot(metrics_q["horizon_days"], metrics_q["val_p10p90_coverage"], marker="o")
plt.title("P10–P90 Interval Coverage vs Horizon (VAL)")
plt.xlabel("Horizon (days ahead)")
plt.ylabel("Coverage")
plt.ylim(0,1)
save_fig(os.path.join(OUT_DIR, "coverage_vs_horizon.png"))

plt.figure(figsize=(8,3))
plt.plot(metrics_q["horizon_days"], metrics_q["val_p10p90_width"], marker="o")
plt.title("Mean Interval Width (P90–P10) vs Horizon (VAL)")
plt.xlabel("Horizon (days ahead)")
plt.ylabel("Width (Euros)")
save_fig(os.path.join(OUT_DIR, "interval_width_vs_horizon.png"))

# -------------------------
# H) Forecast table (next 7 days) for sample stores + save CSV
# -------------------------
stores_unique = np.array(sorted(df_all["Store"].unique()))
rng = np.random.default_rng(42)
sample_stores = rng.choice(stores_unique, size=min(SAMPLE_STORES_N, len(stores_unique)), replace=False)

if FORECAST_ORIGIN_MODE == "before_test":
    origin_date = test_start - pd.Timedelta(days=1)
elif FORECAST_ORIGIN_MODE == "custom":
    origin_date = pd.to_datetime(CUSTOM_ORIGIN_DATE)
else:
    origin_date = test_start - pd.Timedelta(days=1)

base = df_all[df_all["Date"] == origin_date].copy()
base = base[base["Store"].isin(sample_stores)].copy()
base = base.dropna(subset=feature_cols)

X0 = base[feature_cols].to_numpy(dtype=np.float32)
stores0 = base["Store"].to_numpy(dtype=int)

forecast_rows = []
for h in HORIZONS:
    fdate = origin_date + pd.Timedelta(days=h)
    p10 = models_q[h][0.1].predict(X0)
    p50 = models_q[h][0.5].predict(X0)
    p90 = models_q[h][0.9].predict(X0)
    
    p10_real = np.expm1(p10)
    p50_real = np.expm1(p50)
    p90_real = np.expm1(p90)
    
    for i in range(len(stores0)):
        forecast_rows.append({
            "Store": int(stores0[i]),
            "OriginDate": origin_date.date().isoformat(),
            "ForecastDate": fdate.date().isoformat(),
            "h": int(h),
            "p10": float(p10_real[i]),
            "p50": float(p50_real[i]),
            "p90": float(p90_real[i]),
        })

forecast_tbl = pd.DataFrame(forecast_rows)
save_df_csv(os.path.join(OUT_DIR, "forecast_sample_next_horizons.csv"), forecast_tbl)
display(forecast_tbl.head(7))

# -------------------------
# I) Ensemble uncertainty (optional) - on day 7
# -------------------------
def train_ensemble_median(h, K=7):
    m_tr, m_va, _ = split_masks_for_h(df_all, h)
    Xtr, ytr, _ = build_xy_for_h(df_all, h, m_tr)
    Xva, yva, _ = build_xy_for_h(df_all, h, m_va)

    ens = []
    for k in range(K):
        mdl = QuantileLinearGD(q=0.5, lr=Q_LR, epochs=max(10, Q_EPOCHS-3), batch_size=Q_BATCH, l2=Q_L2, seed=100+k, verbose=0)
        mdl.fit(Xtr, ytr, Xva, yva)
        ens.append(mdl)

    preds_log = np.stack([m.predict(Xva) for m in ens], axis=0)
    preds_real = np.expm1(preds_log)
    
    mu_real = preds_real.mean(axis=0)
    sd_real = preds_real.std(axis=0)
    yva_real = np.expm1(yva)

    z = 1.6449  # ~90% interval
    lo = mu_real - z*sd_real
    hi = mu_real + z*sd_real
    coverage = float(np.mean((yva_real >= lo) & (yva_real <= hi)))
    
    return (mu_real, sd_real, lo, hi, yva_real, coverage, float(np.sqrt(np.mean((yva_real - mu_real) ** 2))))

if DO_ENSEMBLE and (ENSEMBLE_HORIZON in HORIZONS):
    mu, sd, lo, hi, yva_e, cov_e, rmse_e = train_ensemble_median(ENSEMBLE_HORIZON, K=ENSEMBLE_K)
    ens_metrics = pd.DataFrame([{
        "horizon_days": ENSEMBLE_HORIZON,
        "K": ENSEMBLE_K,
        "val_rmse_mean": rmse_e,
        "val_coverage_approx90": cov_e,
        "val_mean_std": float(np.mean(sd)),
        "val_mean_interval_width": float(np.mean(hi-lo)),
    }])
    save_df_csv(os.path.join(OUT_DIR, "metrics_ensemble_uncertainty.csv"), ens_metrics)
    display(ens_metrics)

    plt.figure(figsize=(6,3))
    plt.hist(sd, bins=30)
    plt.title(f"Ensemble predictive std distribution (VAL) — h={ENSEMBLE_HORIZON}")
    plt.xlabel("std (Euros)")
    plt.ylabel("count")
    save_fig(os.path.join(OUT_DIR, f"ensemble_std_hist_h{ENSEMBLE_HORIZON}.png"))

# -------------------------
# J) Classification (low/med/high) + ROC-AUC + F1 (OvR)
# -------------------------
def make_class_labels(y_real, t1, t2):
    y_real = np.asarray(y_real, dtype=np.float64)
    cls = np.zeros_like(y_real, dtype=np.int64)
    cls[(y_real > t1) & (y_real <= t2)] = 1
    cls[y_real > t2] = 2
    return cls

def softmax(z):
    z = z - np.max(z, axis=1, keepdims=True)
    e = np.exp(z)
    return e / (np.sum(e, axis=1, keepdims=True) + 1e-12)

class SoftmaxRegressionGD:
    def __init__(self, n_classes=3, lr=0.2, epochs=22, batch_size=8192, l2=1e-4, seed=42, verbose=1):
        self.n_classes=n_classes; self.lr=lr; self.epochs=epochs; self.batch_size=batch_size
        self.l2=l2; self.seed=seed; self.verbose=verbose
        self.W=None; self.b=None

    def fit(self, X, y, X_val=None, y_val=None):
        n, d = X.shape
        k = self.n_classes
        rng = np.random.default_rng(self.seed)
        self.W = rng.normal(0, 0.01, size=(d, k)).astype(np.float32)
        self.b = np.zeros((k,), dtype=np.float32)

        best = (float("inf"), self.W.copy(), self.b.copy())

        def one_hot(y, k):
            oh = np.zeros((len(y), k), dtype=np.float32)
            oh[np.arange(len(y)), y] = 1.0
            return oh

        for ep in range(1, self.epochs+1):
            idx = rng.permutation(n)
            Xs, ys = X[idx], y[idx]

            for i in range(0, n, self.batch_size):
                xb = Xs[i:i+self.batch_size]
                yb = ys[i:i+self.batch_size]
                Y = one_hot(yb, k)

                logits = xb @ self.W + self.b
                P = softmax(logits)

                G = (P - Y) / len(xb)
                gW = xb.T @ G
                gb = np.sum(G, axis=0)

                if self.l2 > 0:
                    gW = gW + self.l2 * self.W

                self.W -= self.lr * gW
                self.b -= self.lr * gb

            if X_val is not None:
                Pv = self.predict_proba(X_val)
                ce = -np.mean(np.log(Pv[np.arange(len(y_val)), y_val] + 1e-12))
                if ce < best[0]:
                    best = (ce, self.W.copy(), self.b.copy())
                if self.verbose:
                    print(f"Epoch {ep:02d} | Val CE={ce:.4f}")

        if X_val is not None:
            _, self.W, self.b = best
        return self

    def predict_proba(self, X):
        return softmax(X @ self.W + self.b).astype(np.float32)

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

def f1_macro(y_true, y_pred, n_classes=3):
    y_true = np.asarray(y_true, dtype=np.int64)
    y_pred = np.asarray(y_pred, dtype=np.int64)
    f1s = []
    for c in range(n_classes):
        tp = np.sum((y_true==c) & (y_pred==c))
        fp = np.sum((y_true!=c) & (y_pred==c))
        fn = np.sum((y_true==c) & (y_pred!=c))
        prec = tp / (tp + fp + 1e-12)
        rec  = tp / (tp + fn + 1e-12)
        f1 = 2*prec*rec / (prec + rec + 1e-12)
        f1s.append(f1)
    return float(np.mean(f1s))

def binary_auc_roc(y_true_bin, y_score):
    y_true_bin = np.asarray(y_true_bin, dtype=np.int64)
    y_score = np.asarray(y_score, dtype=np.float64)
    pos = (y_true_bin == 1)
    neg = (y_true_bin == 0)
    n_pos = int(np.sum(pos)); n_neg = int(np.sum(neg))
    if n_pos == 0 or n_neg == 0:
        return np.nan

    ranks = pd.Series(y_score).rank(method="average").to_numpy(dtype=np.float64)
    sum_ranks_pos = float(np.sum(ranks[pos]))
    auc = (sum_ranks_pos - n_pos*(n_pos+1)/2.0) / (n_pos*n_neg)
    return float(auc)

def auc_roc_ovr_macro(y_true, proba, n_classes=3):
    y_true = np.asarray(y_true, dtype=np.int64)
    proba = np.asarray(proba, dtype=np.float64)
    aucs = []
    for c in range(n_classes):
        y_bin = (y_true == c).astype(int)
        auc = binary_auc_roc(y_bin, proba[:, c])
        if not np.isnan(auc):
            aucs.append(auc)
    return float(np.mean(aucs)) if len(aucs) else np.nan

def confusion_matrix(y_true, y_pred, n_classes=3):
    cm = np.zeros((n_classes, n_classes), dtype=int)
    for t,p in zip(y_true, y_pred):
        cm[int(t), int(p)] += 1
    return cm

clf_rows = []

for h in CLASS_HORIZONS:
    if h > H_FULL:
        continue

    m_tr, m_va, m_te = split_masks_for_h(df_all, h)
    Xtr, ytr, _ = build_xy_for_h(df_all, h, m_tr)
    Xva, yva, _ = build_xy_for_h(df_all, h, m_va)
    Xte, yte, _ = build_xy_for_h(df_all, h, m_te)

    ytr_real = np.expm1(ytr)
    yva_real = np.expm1(yva)
    yte_real = np.expm1(yte)

    t1 = np.quantile(ytr_real, 1/3)
    t2 = np.quantile(ytr_real, 2/3)

    ytr_c = make_class_labels(ytr_real, t1, t2)
    yva_c = make_class_labels(yva_real, t1, t2)
    yte_c = make_class_labels(yte_real, t1, t2)

    print(f"\n=== Classification horizon h={h} ===")
    print(f"Thresholds (Euros): Low < {t1:.1f} <= Med <= {t2:.1f} < High")
    print("Train class counts:", np.bincount(ytr_c, minlength=3))

    clf = SoftmaxRegressionGD(n_classes=3, lr=CLF_LR, epochs=CLF_EPOCHS, batch_size=CLF_BATCH, l2=CLF_L2, verbose=0)
    clf.fit(Xtr, ytr_c, Xva, yva_c)

    Pva = clf.predict_proba(Xva)
    Pte = clf.predict_proba(Xte)
    yva_pred = np.argmax(Pva, axis=1)
    yte_pred = np.argmax(Pte, axis=1)

    val_f1 = f1_macro(yva_c, yva_pred, 3)
    val_auc = auc_roc_ovr_macro(yva_c, Pva, 3)
    test_f1 = f1_macro(yte_c, yte_pred, 3)
    test_auc = auc_roc_ovr_macro(yte_c, Pte, 3)

    clf_rows.append({
        "horizon_days": h,
        "t1": float(t1),
        "t2": float(t2),
        "val_f1_macro": val_f1,
        "val_auc_ovr_macro": val_auc,
        "test_f1_macro": test_f1,
        "test_auc_ovr_macro": test_auc,
    })

    cm_val = confusion_matrix(yva_c, yva_pred, 3)
    cm_test = confusion_matrix(yte_c, yte_pred, 3)

    plt.figure(figsize=(4,3))
    plt.imshow(cm_val, aspect="auto")
    plt.title(f"Confusion Matrix (VAL) — h={h}")
    plt.xlabel("Pred"); plt.ylabel("True")
    plt.colorbar()
    save_fig(os.path.join(OUT_DIR, f"cm_val_h{h}.png"))

    plt.figure(figsize=(4,3))
    plt.imshow(cm_test, aspect="auto")
    plt.title(f"Confusion Matrix (TEST) — h={h}")
    plt.xlabel("Pred"); plt.ylabel("True")
    plt.colorbar()
    save_fig(os.path.join(OUT_DIR, f"cm_test_h{h}.png"))

clf_metrics = pd.DataFrame(clf_rows).sort_values("horizon_days")
save_df_csv(os.path.join(OUT_DIR, "metrics_classification.csv"), clf_metrics)
display(clf_metrics)

# -------------------------
# K) Quick summary text file
# -------------------------
summary_lines = []
summary_lines.append("Phase 4 outputs saved in artifacts/phase4/")
summary_lines.append(f"Horizons trained: {HORIZONS}")
summary_lines.append("Files:")
summary_lines.append("- metrics_multistep_quantile.csv")
summary_lines.append("- rmspe_vs_horizon.png")
summary_lines.append("- coverage_vs_horizon.png")
summary_lines.append("- interval_width_vs_horizon.png")
summary_lines.append("- forecast_sample_next_horizons.csv")
summary_lines.append("- metrics_classification.csv")
if DO_ENSEMBLE and (ENSEMBLE_HORIZON in HORIZONS):
    summary_lines.append("- metrics_ensemble_uncertainty.csv")
    summary_lines.append(f"- ensemble_std_hist_h{ENSEMBLE_HORIZON}.png")
for h in CLASS_HORIZONS:
    summary_lines.append(f"- cm_val_h{h}.png / cm_test_h{h}.png")

with open(os.path.join(OUT_DIR, "SUMMARY.txt"), "w", encoding="utf-8") as f:
    f.write("\n".join(summary_lines))

print("\n".join(summary_lines))

Split meta: {'max_date': '2015-07-31', 'val_start': '2015-05-09', 'test_start': '2015-06-20', 'train_rows': 923549, 'val_rows': 46830, 'test_rows': 46830, 'test_days': 42, 'val_days': 42}
Total rows: 1017209 | Num features: 59

=== Horizon h=1 days ===
Train/Val/Test: (837426, 59) (2043, 59) (1992, 59)

=== Horizon h=2 days ===
Train/Val/Test: (835299, 59) (1896, 59) (1848, 59)

=== Horizon h=3 days ===
Train/Val/Test: (833172, 59) (1753, 59) (1702, 59)

=== Horizon h=4 days ===
Train/Val/Test: (831045, 59) (1612, 59) (1566, 59)

=== Horizon h=5 days ===
Train/Val/Test: (828918, 59) (1480, 59) (1436, 59)

=== Horizon h=6 days ===
Train/Val/Test: (826791, 59) (1352, 59) (1310, 59)

=== Horizon h=7 days ===
Train/Val/Test: (824664, 59) (1234, 59) (1192, 59)


,horizon_days,val_rmse_p50,val_rmspe_p50,val_mape_p50,val_r2_p50,val_pinball_p10,val_pinball_p50,val_pinball_p90,val_p10p90_coverage,val_p10p90_width,test_rmse_p50,test_rmspe_p50,test_mape_p50,test_r2_p50
0,1,6731.161133,0.999995,0.999995,-2.463251,567.685242,2838.398926,5109.112793,0.0,0.0,6992.097168,0.999997,0.999997,-2.906716
1,2,6792.583008,0.999996,0.999996,-2.657975,579.024292,2895.084473,5211.145020,0.0,0.0,7079.646973,0.999998,0.999998,-3.166268
2,3,6802.216309,0.999995,0.999995,-2.622007,578.760010,2893.769531,5208.778809,0.0,0.0,7126.923340,0.999998,0.999998,-3.123093
3,4,6787.185059,0.999996,0.999996,-2.565333,575.729919,2878.608643,5181.486816,0.0,0.0,7137.979980,0.999998,0.999998,-3.127309
4,5,6835.941406,0.999996,0.999996,-2.550572,579.393860,2896.936523,5214.479004,0.0,0.0,7157.231934,0.999998,0.999998,-3.086039
5,6,6910.300293,0.999995,0.999995,-2.789315,592.884033,2964.395996,5335.907715,0.0,0.0,7186.795898,0.999998,0.999998,-3.032157
6,7,6905.974121,0.999996,0.999996,-2.766487,591.871277,2959.319336,5326.767578,0.0,0.0,7163.399414,0.999998,0.999998,-3.018601


""


,horizon_days,K,val_rmse_mean,val_coverage_approx90,val_mean_std,val_mean_interval_width
0,7,7,6905.995605,0.172609,0.073092,0.240459



=== Classification horizon h=1 ===
Thresholds (Euros): Low < 4498.0 <= Med <= 7031.0 < High
Train class counts: [279215 279105 279106]

=== Classification horizon h=7 ===
Thresholds (Euros): Low < 4500.0 <= Med <= 7033.0 < High
Train class counts: [274957 274904 274803]


,horizon_days,t1,t2,val_f1_macro,val_auc_ovr_macro,test_f1_macro,test_auc_ovr_macro
0,1,4498.000977,7031.002441,0.721119,0.884956,0.785705,0.931242
1,7,4500.000488,7033.000977,0.791585,0.922837,0.815225,0.944688


Phase 4 outputs saved in artifacts/phase4/
Horizons trained: [1, 2, 3, 4, 5, 6, 7]
Files:
- metrics_multistep_quantile.csv
- rmspe_vs_horizon.png
- coverage_vs_horizon.png
- interval_width_vs_horizon.png
- forecast_sample_next_horizons.csv
- metrics_classification.csv
- metrics_ensemble_uncertainty.csv
- ensemble_std_hist_h7.png
- cm_val_h1.png / cm_test_h1.png
- cm_val_h7.png / cm_test_h7.png
